# Initial Imports & Setup

In [2]:
# Data Science Modules
import numpy as np
import pandas as pd
import altair as alt

# Python Modules
import glob as glob
import shutil
import os
import datetime as datetime

# Lookup Data

* **./files/names.csv**: CSV File with Provider/Clinic Names, etc. Names is used to determine active providers/clinics. (So if there's no name, it wont get evaluated later -- say, for example, after Everson Closes.)
* **./files/metrics.csv**: CSV File with Metrics information (Display Name, Category, Target, etc.)

In [4]:
# Read Lookup files into dataframes for mapping
names = pd.read_csv('./files/names.csv', index_col='MeridiosName')
metrics = pd.read_csv('./files/metrics.csv', index_col='MeridiosMetric', dtype={'MeridiosMetric':object})

# Import Data from /data folder.

* **./data/(Zero-Padded Date at beginning of CSVs from Meridos).csv**: Files from Automatic weekly report from Meridios

In [5]:
# The base dataframe, df, get's transformed into different data subsets. 
df = pd.DataFrame()

# Read in all the data files and append to dataframe
files = glob.glob('./data/*.csv')
for file in files:
    filedf = pd.read_csv(file,
                        usecols=["NAME","Metricname","SeenNum","SeenDenom"])
    # Prettier Names but we'll drop these eventually.
    filedf = filedf.rename(columns={'NAME': 'MeridiosName',
                                    'Metricname': 'MeridiosMetric',
                                    'SeenNum': 'Numerator', 
                                    'SeenDenom': 'Denominator'})

    # Add Lookup Columns.
    filedf['Name'] = filedf.MeridiosName.map(names.Name)
    filedf['Type'] = filedf.MeridiosName.map(names.Type)
    filedf['Clinic'] = filedf.MeridiosName.map(names.Clinic)
    filedf['Metric'] = filedf.MeridiosMetric.map(metrics.Metric)
    
    # Who are we kidding with precision? Round to Basis points
    # Not actually percentage (but rather than * 100 leaving for formatting
    # in visualization.)
    filedf['Percentage'] = round(filedf['Numerator'] / filedf['Denominator'], 4)

    # Zero Padded Dates with Dots. Like: "03.15.2018" We're using the filename
    # as the source of the date column because Meridios exports haven't always 
    # had the date-time data correct. 
    filename_parts = file[7:-4].split(' ')
    if (len(str(filename_parts[0])) is 10):
        filedf['Date'] = datetime.datetime.strptime(filename_parts[0], "%m.%d.%Y")

        # Now delete the 'raw' columns from import
        filedf.drop(["MeridiosName","MeridiosMetric","Numerator","Denominator"], axis=1, inplace=True)

        # We're only appending data that had Zero Padded Date at 
        # beginning of filename.
        df = df.append(filedf) 
    else:
        print("CSV Data File doesn't have Zero-Padded Date at beginning.")
        
# We can inspect dataframe different ways        
#df.dtypes
#df.info()
#df.describe()
#df.columns
#df.Metric.unique()
#df.head()
#metrics.head()

## Validations:

* Do new CSV files include new MeridiosName, New Metric items? (Has the underlying report changed? Such as when edited provider name.)
* Should we keep a list of already imported data and do some message to clarify after new data is available?
* Percentages under 0% and over 100% are report errors

Currently no validations. But probably should automate checks that new data from report is accurate. 

# Altair Graphs

make_individual_metric_chart makes a layered chart with the individuals recent progress, their clinic's progress, and how it's currently distributed in FCN. Assumes that dataframes names & metrics exists.  

In [6]:
def make_individual_metric_chart(metric, name):
    # Data Transformations in Pandas for graphs.
    # assumes dataframe 'df' that has data from CSVs
    # assumes dataframes 'names' and 'metrics' for lookups
    providerdf = df[(df['Metric']==metric) & 
                    (df['Type']=='Individual') &
                    (df['Name']==name)]
    
    clinic_name = names[names.Name == name].iloc[0].Clinic

    clinicdf = df[(df['Metric']==metric) & 
                    (df['Name']==clinic_name)]

    combodf = pd.concat([providerdf, clinicdf])

    current_date = max(df['Date'])
    current_metric = df[(df['Metric']==metric) & 
                        (df['Type']=='Individual') &
                        (df['Date']==current_date)]

    # Altair Graphs to combine.
    progress_line = alt.Chart(combodf
        ).mark_line(
        ).encode(
            alt.X('Date:T', title=""),
            alt.Y('Percentage:Q', axis=alt.Axis(format="%", title=""), 
                  scale=alt.Scale(domain=(0, 1))),
            alt.Color('Name:N', legend=alt.Legend(title="", orient="bottom-left"))
        ).properties(
            width=200,
            height=200,
        )

    fcn_current_strip_chart = alt.Chart(current_metric
        ).mark_tick(
            color='#eee',
        ).encode(
            alt.Y('Percentage:Q', axis=alt.Axis(format="%", title="", labels=False,),
                 scale=alt.Scale(domain=(0, 1))),
        ).properties(
            height=200,
        )


    provider_highlight_strip = alt.Chart(current_metric
        ).mark_tick(
        ).encode(
            alt.Y('Percentage:Q')
        ).transform_filter(
        alt.FieldEqualPredicate(field='Name', equal=name)
        )

    provider_percent = provider_highlight_strip.mark_text(
        align='left',
        baseline='middle',
        dx=15,
        size=20,
        ).encode(
            text=alt.Text('Percentage:Q', format='.2%')
        ).transform_filter(
        alt.FieldEqualPredicate(field='Name', equal=name)
        )

    chart = progress_line | (fcn_current_strip_chart + provider_highlight_strip + provider_percent)
    return chart

In [ ]:
# Need to just do active individuals

for name in df.Name.unique():
    for metric in df.Metric.unique():
        chart = make_individual_metric_chart(metric, name) 
        foldername = str(name).replace(" ", "_")
        if not os.path.exists("./altair-output/" + foldername):
            os.makedirs("./altair-output/" + foldername)
        chart.save("./altair-output/" + foldername + '/'+ str(metric).replace(" ", "_") + ".json")
        
    with open('./files/index.html', 'r') as file :
        filedata = file.read()
    filedata = filedata.replace('{{{Provider}}}', name)
    with open('./altair-output/' + foldername + '/index.html', 'w+') as file:
        file.write(filedata)